# Cleaning of OMDB Database

In [399]:
import pandas as pd
import numpy as np

In [400]:
movies = pd.read_csv('../data/preprocessed/movies_id_updated.csv', sep=',')
ratings = pd.read_csv('../data/raw/ratings.csv', sep=',')
omdb = pd.read_csv('../data/raw/omdb_total.csv')

In [401]:
movies = movies.rename(columns = {'id':'movieID'})
rated_movies = pd.merge(movies, ratings, how = 'outer', on = 'movieID')

In [402]:
rated_movies.head()

,movieID,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtPictureURL,user_id,rating
0,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,1339.0,5.0
1,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,551.0,3.5
2,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,336.0,4.5
3,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,1087.0,3.5
4,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,1598.0,4.0


In [403]:
omdb = omdb.drop(['Production','Website','totalSeasons','Season','Episode','seriesID','Type','BoxOffice','DVD','Response','Poster'],axis=1)

In [404]:
movie_data = pd.merge(rated_movies, omdb, how = 'outer', on = 'imdbID')

In [405]:
movie_data = movie_data.drop(['Unnamed: 0'],axis=1)

In [406]:
movie_data.isna().sum()

movieID                        5
title                          5
imdbID                         0
spanishTitle                   5
imdbPictureURL              2230
year                         385
rtID                       15728
rtPictureURL                 385
user_id                      151
rating                       151
Title                        270
Year                         270
Rated                       4401
Released                     958
Runtime                      287
Genre                        282
Director                    3016
Writer                      6282
Actors                       531
Plot                         506
Language                     563
Country                      324
Awards                     48118
Metascore                  94138
imdbRating                   274
imdbVotes                   1903
Internet Movie Database      274
Rotten Tomatoes            32420
Metacritic                 94132
dtype: int64

In [407]:
#Remove '%' and convert to float
movie_data['Rotten Tomatoes'] = movie_data['Rotten Tomatoes'].str.replace(r'\D', '').astype(float)

In [408]:
movie_data.iloc[1, 27]

100.0

In [409]:
movie_data['Rotten Tomatoes'].describe()

count    970054.000000
mean         71.665898
std          23.948137
min           0.000000
25%          57.000000
50%          79.000000
75%          91.000000
max         100.000000
Name: Rotten Tomatoes, dtype: float64

In [410]:
# Convert 'Rotten Tomatoes Score to [0.5 ; 5]'
# Set to 0.5 if a 0 was given as score -> leads to a slightly better score after conversion

movie_data['Rotten Tomatoes'] = movie_data['Rotten Tomatoes'].apply(lambda x: x/20).apply(lambda x: 0.5 if x < 0.5 else x)

In [411]:
#Replace '/10' and convert to float
movie_data['Internet Movie Database'] = movie_data['Internet Movie Database'].str.replace(r'/10', '').astype(float)

In [412]:
movie_data['Internet Movie Database']

0          8.3
1          8.3
2          8.3
3          8.3
4          8.3
          ... 
1002469    7.2
1002470    NaN
1002471    NaN
1002472    NaN
1002473    NaN
Name: Internet Movie Database, Length: 1002474, dtype: float64

In [413]:
movie_data['Internet Movie Database'].describe()

count    1.002200e+06
mean     7.141970e+00
std      9.221005e-01
min      1.500000e+00
25%      6.600000e+00
50%      7.300000e+00
75%      7.800000e+00
max      9.400000e+00
Name: Internet Movie Database, dtype: float64

In [414]:
# Convert 'Internet Movie Database' Score to [0.5 ; 5]'
# The Internet Movie Database Score is already an average which lies in [1,10]
# Doing conversion based on this assumption -> no movie gets 0.5 as score (since there was no 1)

movie_data['Internet Movie Database'] = movie_data['Internet Movie Database'].apply(lambda x: x/2).apply(lambda x: 0.5 if x < 0.5 else x)

In [415]:
# Convert 'imdbRating' Score to [0.5 ; 5]'
# imdbRating seems to be the same as 'Internet Movie Database' Score -> keep in mind
movie_data['imdbRating'] = movie_data['imdbRating'].apply(lambda x: x/2).apply(lambda x: 0.5 if x < 0.5 else x)

In [416]:
#Replace '/100' and convert to float
movie_data['Metacritic'] = movie_data['Metacritic'].str.replace(r'/100', '').astype(float)

In [417]:
movie_data['Metacritic']

0          95.0
1          95.0
2          95.0
3          95.0
4          95.0
           ... 
1002469    69.0
1002470     NaN
1002471     NaN
1002472     NaN
1002473     NaN
Name: Metacritic, Length: 1002474, dtype: float64

In [418]:
movie_data['Metacritic'].describe()

count    908342.000000
mean         65.623761
std          17.431752
min           1.000000
25%          54.000000
50%          66.000000
75%          79.000000
max         100.000000
Name: Metacritic, dtype: float64

In [419]:
# Convert 'Metacritic' Score to [0.5 ; 5]'

movie_data['Metacritic'] = movie_data['Metacritic'].apply(lambda x: x/20).apply(lambda x: 0.5 if x < 0.5 else x)

In [420]:
movie_data['Metascore'].describe()

count    908336.000000
mean         65.623679
std          17.431780
min           1.000000
25%          54.000000
50%          66.000000
75%          79.000000
max         100.000000
Name: Metascore, dtype: float64

In [421]:
# Convert 'Metascore'
movie_data['Metascore'] = movie_data['Metascore'].apply(lambda x: x/20).apply(lambda x: 0.5 if x < 0.5 else x)

In [422]:
# Convert runtime to float and remove 'min'
movie_data['Runtime'] = movie_data['Runtime'].str.replace(r'min', '').astype(float)

In [423]:
movie_data['Runtime'].head()

0    81.0
1    81.0
2    81.0
3    81.0
4    81.0
Name: Runtime, dtype: float64

In [424]:
#Convert date into float
#Not 100% sure if this is entirely correct but stackoverflow said so and it seems right
movie_data['Released'] = pd.to_datetime(movie_data['Released']).dt.strftime("%Y%m%d").astype(float)

In [425]:
movie_data['Released'].describe()

count    1.001516e+06
mean     1.992629e+07
std      1.490312e+05
min      1.903121e+07
25%      1.988080e+07
50%      1.997071e+07
75%      2.003021e+07
max      2.019080e+07
Name: Released, dtype: float64

In [426]:
movie_data.dtypes

movieID                    float64
title                       object
imdbID                      object
spanishTitle                object
imdbPictureURL              object
year                       float64
rtID                        object
rtPictureURL                object
user_id                    float64
rating                     float64
Title                       object
Year                        object
Rated                       object
Released                   float64
Runtime                    float64
Genre                       object
Director                    object
Writer                      object
Actors                      object
Plot                        object
Language                    object
Country                     object
Awards                      object
Metascore                  float64
imdbRating                 float64
imdbVotes                   object
Internet Movie Database    float64
Rotten Tomatoes            float64
Metacritic          

In [427]:
movie_data['Title'].isna().sum()

270

In [428]:
# Remove NaN 'titles' - (Rows of NaN titles are almost completely NaN)
man_rows = pd.DataFrame()
man_rows = movie_data.index[movie_data['title'].isnull()]
print(man_rows)
for i in man_rows:
    movie_data = movie_data.drop([i], axis=0)

Int64Index([1002469, 1002470, 1002471, 1002472, 1002473], dtype='int64')


In [429]:
movie_data['title'].isnull()

0          False
1          False
2          False
3          False
4          False
           ...  
1002464    False
1002465    False
1002466    False
1002467    False
1002468    False
Name: title, Length: 1002469, dtype: bool

In [430]:
movie_data

,movieID,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtPictureURL,user_id,rating,...,Plot,Language,Country,Awards,Metascore,imdbRating,imdbVotes,Internet Movie Database,Rotten Tomatoes,Metacritic
0,1.0,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,1339.0,5.0,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,4.75,4.15,"852,896",4.15,5.00,4.75
1,1.0,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,1339.0,5.0,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,4.75,4.15,"852,896",4.15,5.00,4.75
2,1.0,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,1339.0,5.0,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,4.75,4.15,"852,896",4.15,5.00,4.75
3,1.0,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,551.0,3.5,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,4.75,4.15,"852,896",4.15,5.00,4.75
4,1.0,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,551.0,3.5,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,4.75,4.15,"852,896",4.15,5.00,4.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002464,65126.0,Choke,tt1024715,Choke,http://ia.media-imdb.com/images/M/MV5BMTMxMDI4...,2008.0,choke,http://content6.flixster.com/movie/10/85/09/10...,1273.0,3.0,...,A sex-addicted con-man pays for his mother's h...,English,USA,2 wins & 7 nominations.,2.35,3.20,"31,058",3.20,2.75,2.35
1002465,65126.0,Choke,tt1024715,Choke,http://ia.media-imdb.com/images/M/MV5BMTMxMDI4...,2008.0,choke,http://content6.flixster.com/movie/10/85/09/10...,599.0,3.5,...,A sex-addicted con-man pays for his mother's h...,English,USA,2 wins & 7 nominations.,2.35,3.20,"31,058",3.20,2.75,2.35
1002466,65133.0,Blackadder Back & Forth,tt0212579,Blackadder Back & Forth,http://ia.media-imdb.com/images/M/MV5BMjA5MjU4...,1999.0,blackadder-back-forth,http://content7.flixster.com/movie/34/10/69/34...,1059.0,4.0,...,"At a New Millennium Eve party, Blackadder and ...",English,UK,1 nomination.,NaN,3.85,"13,510",3.85,NaN,NaN
1002467,65133.0,Blackadder Back & Forth,tt0212579,Blackadder Back & Forth,http://ia.media-imdb.com/images/M/MV5BMjA5MjU4...,1999.0,blackadder-back-forth,http://content7.flixster.com/movie/34/10/69/34...,480.0,5.0,...,"At a New Millennium Eve party, Blackadder and ...",English,UK,1 nomination.,NaN,3.85,"13,510",3.85,NaN,NaN


# TODO:
-Remove all other useless rows (with too many NaNs?)

-Remove double/multiple ratings of same users

-Check if all movies are merged under the same omdbID

-Print out data into ../data/preprocessed folder

-Build dataframe/csv with average rating for each omdbID ?

In [ ]:
# Remove Duplicated Ratings

In [340]:
# Export
# Everyone can run this command themselves - i won't check in this file (0,7gb) (added to .gitignore)

movie_data.to_csv('../data/preprocessed/omdb_cleaned.csv')

# Build csv with average rating for each omdbID

In [431]:
movie_data.head()

,movieID,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtPictureURL,user_id,rating,...,Plot,Language,Country,Awards,Metascore,imdbRating,imdbVotes,Internet Movie Database,Rotten Tomatoes,Metacritic
0,1.0,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,1339.0,5.0,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,4.75,4.15,"852,896",4.15,5.0,4.75
1,1.0,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,1339.0,5.0,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,4.75,4.15,"852,896",4.15,5.0,4.75
2,1.0,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,1339.0,5.0,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,4.75,4.15,"852,896",4.15,5.0,4.75
3,1.0,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,551.0,3.5,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,4.75,4.15,"852,896",4.15,5.0,4.75
4,1.0,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...,551.0,3.5,...,A cowboy doll is profoundly threatened and jea...,English,USA,Nominated for 3 Oscars. Another 27 wins & 20 n...,4.75,4.15,"852,896",4.15,5.0,4.75


In [440]:
average_ratings = movie_data

In [441]:
average_ratings = average_ratings.drop(['spanishTitle','imdbPictureURL','Year','year','Rated','Released','Runtime','Genre','Title','Director','Writer','Actors','rtID','rtPictureURL','Plot','Language','Country','Awards','imdbVotes'],axis=1)

In [442]:
average_ratings.describe()

,movieID,user_id,rating,Metascore,imdbRating,Internet Movie Database,Rotten Tomatoes,Metacritic
count,1.002469e+06,1.002323e+06,1.002323e+06,908335.000000,1.002199e+06,1.002199e+06,970050.000000,908341.000000
mean,8.934601e+03,6.928460e+02,3.439775e+00,3.281309,3.570985e+00,3.570985e+00,3.586193,3.281314
std,1.464392e+04,4.989834e+02,9.981290e-01,0.871166,4.610505e-01,4.610505e-01,1.189517,0.871165
min,1.000000e+00,0.000000e+00,5.000000e-01,0.500000,7.500000e-01,7.500000e-01,0.500000,0.500000
25%,1.370000e+03,2.740000e+02,3.000000e+00,2.700000,3.300000e+00,3.300000e+00,2.850000,2.700000
50%,3.274000e+03,5.950000e+02,3.500000e+00,3.300000,3.650000e+00,3.650000e+00,3.950000,3.300000
75%,6.642000e+03,1.045000e+03,4.000000e+00,3.950000,3.900000e+00,3.900000e+00,4.550000,3.950000
max,6.513300e+04,2.112000e+03,5.000000e+00,5.000000,4.700000e+00,4.700000e+00,5.000000,5.000000


In [443]:
average_ratings = average_ratings.sort_values(by=['imdbID'])

In [444]:
avg = average_ratings.groupby('imdbID') ['rating'].mean()
average_ratings = average_ratings.join(avg, on='imdbID', rsuffix='Avg')

imdb_avg = average_ratings.groupby('imdbID')['imdbRating'].mean()
average_ratings = average_ratings.join(imdb_avg, on='imdbID', rsuffix=' Avg')

rt_avg = average_ratings.groupby('imdbID')['Rotten Tomatoes'].mean()
average_ratings = average_ratings.join(rt_avg, on='imdbID', rsuffix='Avg')

iMB_avg = average_ratings.groupby('imdbID')['Internet Movie Database'].mean()
average_ratings = average_ratings.join(iMB_avg, on='imdbID', rsuffix=' Avg')

metascore_avg = average_ratings.groupby('imdbID')['Metascore'].mean()
average_ratings = average_ratings.join(metascore_avg, on='imdbID', rsuffix=' Avg')

metacritic_avg = average_ratings.groupby('imdbID')['Metacritic'].mean()
average_ratings = average_ratings.join(metacritic_avg, on='imdbID', rsuffix=' Avg')

In [445]:
average_ratings.describe()

,movieID,user_id,rating,Metascore,imdbRating,Internet Movie Database,Rotten Tomatoes,Metacritic,ratingAvg,imdbRating Avg,Rotten TomatoesAvg,Internet Movie Database Avg,Metascore Avg,Metacritic Avg
count,1.002469e+06,1.002323e+06,1.002323e+06,908335.000000,1.002199e+06,1.002199e+06,970050.000000,908341.000000,1.002354e+06,1.002199e+06,970050.000000,1.002199e+06,908335.000000,908341.000000
mean,8.934601e+03,6.928460e+02,3.439775e+00,3.281309,3.570985e+00,3.570985e+00,3.586193,3.281314,3.439773e+00,3.570985e+00,3.586193,3.570985e+00,3.281309,3.281314
std,1.464392e+04,4.989834e+02,9.981290e-01,0.871166,4.610505e-01,4.610505e-01,1.189517,0.871165,4.717395e-01,4.610505e-01,1.189517,4.610505e-01,0.871166,0.871165
min,1.000000e+00,0.000000e+00,5.000000e-01,0.500000,7.500000e-01,7.500000e-01,0.500000,0.500000,5.000000e-01,7.500000e-01,0.500000,7.500000e-01,0.500000,0.500000
25%,1.370000e+03,2.740000e+02,3.000000e+00,2.700000,3.300000e+00,3.300000e+00,2.850000,2.700000,3.160000e+00,3.300000e+00,2.850000,3.300000e+00,2.700000,2.700000
50%,3.274000e+03,5.950000e+02,3.500000e+00,3.300000,3.650000e+00,3.650000e+00,3.950000,3.300000,3.507042e+00,3.650000e+00,3.950000,3.650000e+00,3.300000,3.300000
75%,6.642000e+03,1.045000e+03,4.000000e+00,3.950000,3.900000e+00,3.900000e+00,4.550000,3.950000,3.788779e+00,3.900000e+00,4.550000,3.900000e+00,3.950000,3.950000
max,6.513300e+04,2.112000e+03,5.000000e+00,5.000000,4.700000e+00,4.700000e+00,5.000000,5.000000,5.000000e+00,4.700000e+00,5.000000,4.700000e+00,5.000000,5.000000


In [446]:
average_ratings = average_ratings.drop(['user_id','rating','rating','Metascore','imdbRating','Internet Movie Database', 'Rotten Tomatoes', 'Metacritic'],axis=1)

In [447]:
average_ratings = average_ratings.drop_duplicates(subset="imdbID")

In [448]:
average_ratings.head()

,movieID,title,imdbID,ratingAvg,imdbRating Avg,Rotten TomatoesAvg,Internet Movie Database Avg,Metascore Avg,Metacritic Avg
844798,6990.0,The Great Train Robbery,tt0000439,3.696970,3.65,5.00,3.65,NaN,NaN
850743,7065.0,The Birth of a Nation,tt0004972,3.228261,3.20,4.65,3.20,NaN,NaN
859392,7243.0,Intolerance: Love's Struggle Throughout the Ages,tt0006864,3.928571,3.90,4.85,3.90,4.95,4.95
886762,8511.0,The Immigrant,tt0008133,3.750000,3.85,NaN,3.85,NaN,NaN
971742,48374.0,Otets Sergiy,tt0008395,3.000000,3.55,NaN,3.55,NaN,NaN


In [ ]:
# Export
average_ratings.to_csv('../data/preprocessed/average_ratings.csv')